## Data Preparation
***

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, log_loss, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, train_test_split, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

df = pd.read_csv('data/Cardiovascular_Diseases_Risk_Prediction_Dataset.csv')
ordinal = ['General_Health', 'Checkup', 'Age_Category']
numeric = list(df.select_dtypes(exclude=object).columns)
categorical = list(df.select_dtypes(object).columns)
for i in ordinal:
    categorical.remove(i)
categorical.remove('Heart_Disease')

In [2]:
# Splitting the training from the validation data
# Making sure the split is stratefied given teh imbalance of our target variable
y = df['Heart_Disease']
X = df.drop('Heart_Disease',axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=12, stratify= y)

In [3]:
print('Training Target Distribution')
print(y_train.value_counts(normalize=True))
print('')
print('Validation Target Distribution')
print(y_test.value_counts(normalize=True))

Training Target Distribution
No     0.91915
Yes    0.08085
Name: Heart_Disease, dtype: float64

Validation Target Distribution
No     0.919147
Yes    0.080853
Name: Heart_Disease, dtype: float64


In [4]:
# Transforming the target variable into 1's and 0's
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [5]:
# Listing categories in order for each ordinal variable.
health = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']
check = [
    'Never', '5 or more years ago', 'Within the past 5 years',
    'Within the past 2 years', 'Within the past year'
]
age = [
    '18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
    '60-64', '65-69', '70-74', '75-79', '80+'
]

# Instantiating an OrdinalEncoder transformer to encode ordinal variables. 
oe = OrdinalEncoder(categories=[health, check, age])


In [6]:
# Instantiating a OneHotEncoder transformer to be used on the categorical varaibles. 
ohe = OneHotEncoder()

In [7]:
# Creating a column transformer to be used in a pipeline
col_transformer = ColumnTransformer(transformers=[
    ('oe', OrdinalEncoder(categories=[health, check, age]), ordinal),
    ('ohe', OneHotEncoder(), categorical)
], remainder="passthrough")